# Structures analysis

Files:
- LTS_CONFCNT_ONLY.OUT : contains the classes and structures in terms of substituted atoms

In [180]:
import sys
sys.path.insert(1, '../../crystal-code-tools/crystal_functions/crystal_functions/')

from file_readwrite import Crystal_output, write_cry_properties, Crystal_input
from convert import cry_gui2pmg

from pymatgen.io.ase import AseAtomsAdaptor
from ase.visualize import view

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.surface import SlabGenerator
import numpy as np
import pandas as pd

In [124]:
path = '/Users/brunocamino/Desktop/Imperial/cmsg_icl/solid-solutions/data/'
file = 'LTS_CONFCNT_ONLY'

new_atom = 'Li'

### Read the structures from the output files and replace with new_atom
This cell generates the structures list. This contains all the 4023 structures as pymatgen structure objects

In [125]:
cry_output = Crystal_output(path+file)
cry_output.config_analysis()

original_structure = cry_gui2pmg(path+file)
structuress = []
ti_atoms = []
for substitutions in cry_output.atom_type2:
    new_structure = original_structure.copy()
    for i in substitutions:
        new_structure.replace(i-1,new_atom)
    structuress.append(new_structure)
    ti_atoms.append((np.array(cry_output.atom_type1[i])-1).tolist())



In [237]:
view(AseAtomsAdaptor().get_atoms(structures[300]))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/cc/bin/p...>

In [126]:
#All structures
structures = structuress

In [7]:
#Random structures
'''structures = []
ti_atoms = []
for i in np.random.choice(4022,size = 100, replace=False):
    structures.append(structuress[i])
    ti_atoms.append((np.array(cry_output.atom_type1[i])-1).tolist())'''

'structures = []\nti_atoms = []\nfor i in np.random.choice(4022,size = 100, replace=False):\n    structures.append(structuress[i])\n    ti_atoms.append((np.array(cry_output.atom_type1[i])-1).tolist())'

In [8]:
#First 100 structures
'''structures = []
ti_atoms = []
for i in range(100):
    structures.append(structuress[i])
    ti_atoms.append((np.array(cry_output.atom_type1[i])-1).tolist())'''

'structures = []\nti_atoms = []\nfor i in range(100):\n    structures.append(structuress[i])\n    ti_atoms.append((np.array(cry_output.atom_type1[i])-1).tolist())'

### Specific structures

In [2]:
layers = cry_gui2pmg('data/structures/Layers.f34')
ordered = cry_gui2pmg('data/structures/Ordered.f34')
dispersed = cry_gui2pmg('data/structures/Dispersed.f34')
structures = [dispersed,ordered,layers]
ti_atoms = []
for structure in structures:
    ti = []
    for i,atom in enumerate(structure.atomic_numbers):
        if atom == 22:
            ti.append(i)
    ti_atoms.append(ti)

### Space group

In [3]:
space_group = []
for structure in structures:
    space_group.append(SpacegroupAnalyzer(structure).get_space_group_number())

In [4]:
space_group

[5, 8, 139]

### Shells

In [5]:
shells = np.unique(np.round(structures[0].distance_matrix[0],decimals=6)).tolist()
shells

[0.0, 2.54275, 3.595992, 4.404172, 5.0855, 5.685762, 6.22844, 7.62825]

### Average Ti-Ti distance

In [6]:
average_ti = []
for n,structure in enumerate(structures):
    average = 0.
    for i in ti_atoms[n]:
        for j in ti_atoms[n]: 
            average += round(structure.sites[i].distance(structure.sites[j]),6) 
    average_ti.append(average/(len(ti_atoms)**2))

In [7]:
average_ti

[40.44476355555557, 36.911891555555584, 34.725968000000016]

### Average second coordination 
The number of Ti atoms in the second shell averaged over all Ti atoms

In [8]:
second_coord_sum = []
for n,structure in enumerate(structures):
    sum_coord = 0
    for i in ti_atoms[n]:
        for j in ti_atoms[n]:
            if round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[2],6):
                sum_coord += 1
    second_coord_sum.append(sum_coord)
second_coord_average = (np.array(second_coord_sum)/len(ti_atoms[n])).tolist()


In [9]:
second_coord_average

[2.888888888888889, 3.5555555555555554, 4.0]

In [10]:
np.unique(np.array(second_coord_average))

array([2.88888889, 3.55555556, 4.        ])

### Average second and third coordination 
The number of Ti atoms in the second and third shells averaged over all Ti atoms

In [11]:
second_third_coord_sum = []
for n,structure in enumerate(structures):
    sum_coord = 0
    for i in ti_atoms[n]:
        for j in ti_atoms[n]:
            if round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[2],6) or \
             round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[4],6):
                sum_coord += 1
    second_third_coord_sum.append(sum_coord)       
second_third_coord_average = (np.array(second_third_coord_sum)/len(ti_atoms[n])).tolist()

In [12]:
np.unique(np.array(second_third_coord_average))

array([4.44444444, 6.66666667, 8.        ])

### Max and min coordination & Ti distribution

#### Second shell

In [281]:
second_coord_max = []
second_coord_min = []
ti_second_shell_atom = []
n_atoms_second_shell = []
for n,structure in enumerate(structures):
    distance = []
    distance_atoms = []
    for i in ti_atoms[n]:
        dist = 0
        dist_atom = []
        for j in ti_atoms[n]:
            if round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[2],6):
                #print(i,j,structure.sites[i].distance(structure.sites[j]))
                dist += 1
                dist_atom.append(j)
        distance_atoms.append(dist_atom)
        #print(i,j,distance_atoms)
        distance.append(dist)
        #print(i,j,distance)
        max_coord_atoms = max(enumerate(distance_atoms), key = lambda tup: len(tup[1]))[1]
        atom_coords = []
        '''for atom in max_coord_atoms:
            atom_coords.append(structure.cart_coords[atom].tolist())'''
    n_atoms_second_shell.append(distance)
    ti_second_shell_atom.append(distance_atoms)
    second_coord_max.append(max(distance))
    second_coord_min.append(min(distance))


In [309]:
unique_coord, first_occ, total_occ = np.unique(np.sort(np.array(n_atoms_second_shell),axis=1),axis=0,return_counts=True,return_index=True)
display(unique_coord)
display(first_occ)
display(total_occ)

array([[2, 3, 4, 5, 5, 5, 5, 5, 6],
       [2, 4, 4, 4, 5, 5, 6, 6, 6],
       [3, 3, 4, 4, 4, 5, 5, 6, 6],
       [3, 3, 4, 4, 5, 5, 5, 5, 6],
       [3, 3, 4, 4, 5, 5, 6, 6, 6],
       [3, 4, 4, 5, 5, 5, 5, 6, 7],
       [4, 4, 5, 5, 5, 5, 6, 6, 6],
       [5, 5, 5, 5, 5, 5, 6, 6, 6]])

array([ 5,  2,  6,  0,  1, 11, 10,  9])

array([  86,    5,   35,    1,  156, 2466,  804,  470])

#### Second shell

In [531]:
fourth_coord_max = []
fourth_coord_min = []
ti_fourth_shell_atom = []
n_atoms_fourth_shell = []
for n,structure in enumerate(structures):
    distance = []
    distance_atoms = []
    for i in ti_atoms[n]:
        dist = 0
        dist_atom = []
        for j in ti_atoms[n]:
            if round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[4],6):
                #print(i,j,structure.sites[i].distance(structure.sites[j]))
                dist += 1
                dist_atom.append(j)
        distance_atoms.append(dist_atom)
        #print(i,j,distance_atoms)
        distance.append(dist)
        #print(i,j,distance)
        max_coord_atoms = max(enumerate(distance_atoms), key = lambda tup: len(tup[1]))[1]
        atom_coords = []
        '''for atom in max_coord_atoms:
            atom_coords.append(structure.cart_coords[atom].tolist())'''
    n_atoms_fourth_shell.append(distance)
    ti_fourth_shell_atom.append(distance_atoms)
    fourth_coord_max.append(max(distance))
    fourth_coord_min.append(min(distance))


In [532]:
np.unique(np.sort(np.array(n_atoms_fourth_shell),axis=1),axis=0,return_counts=True,return_index=True)

(array([[0, 0, 0, 0, 1, 1, 2, 2, 2],
        [0, 0, 0, 1, 1, 1, 1, 2, 2],
        [0, 0, 0, 1, 1, 1, 2, 2, 3],
        [0, 0, 0, 1, 1, 2, 2, 2, 2],
        [0, 0, 0, 2, 2, 2, 2, 2, 2],
        [0, 0, 1, 1, 1, 1, 2, 3, 3]]),
 array([ 1,  0,  2, 11,  9,  3]),
 array([   1,    1,  856, 2466,  470,  229]))

#### Combine second shell and fourth cell

In [550]:
second_fourth_coord_max = []
second_fourth_coord_min = []
ti_second_fourth_shell_atom = []
n_atoms_second_fourth_shell = []
for n,structure in enumerate(structures):
    distance1 = []
    distance2 = []
    distance_atoms = []
    for i in ti_atoms[n]:
        dist1 = 0
        dist2 = 0
        dist1_atom = []
        dist2_atom = []
        for j in ti_atoms[n]:
            if round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[2],6):
                dist1 += 1
                dist1_atom.append(j)
            if round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[4],6):
                dist2 += 1
                dist2_atom.append(j)
        distance_atoms.append(dist1_atom)
        #print(i,j,distance_atoms)
        distance1.append(dist1)
        distance2.append(dist2)
        #print(i,j,distance1)
        max_coord_atoms = max(enumerate(distance_atoms), key = lambda tup: len(tup[1]))[1]
        atom_coords = []
        '''for atom in max_coord_atoms:
            atom_coords.append(structure.cart_coords[atom].tolist())'''
    distance2 = (np.array(distance2)[np.argsort(distance1)]).tolist()
    distance1.sort()
    distance1.extend(distance2)
    n_atoms_second_fourth_shell.append(distance1)
    ti_second_fourth_shell_atom.append(distance_atoms)
    second_fourth_coord_max.append(max(distance))
    second_fourth_coord_min.append(min(distance))

[[3, 3, 4, 4, 5, 5, 5, 5, 6, 2, 2, 1, 1, 1, 1, 0, 0, 0],
 [3, 3, 4, 4, 5, 5, 6, 6, 6, 2, 2, 0, 2, 1, 1, 0, 0, 0],
 [2, 4, 4, 4, 5, 5, 6, 6, 6, 3, 2, 0, 2, 1, 1, 1, 0, 0],
 [3, 3, 4, 4, 5, 5, 6, 6, 6, 3, 3, 0, 2, 1, 1, 1, 1, 0],
 [2, 4, 4, 4, 5, 5, 6, 6, 6, 3, 2, 0, 2, 1, 1, 1, 0, 0],
 [2, 3, 4, 5, 5, 5, 5, 5, 6, 3, 2, 1, 1, 0, 2, 1, 0, 0],
 [3, 3, 4, 4, 4, 5, 5, 6, 6, 3, 2, 1, 0, 2, 1, 0, 1, 0],
 [2, 3, 4, 5, 5, 5, 5, 5, 6, 3, 3, 1, 1, 0, 2, 1, 0, 1],
 [3, 3, 4, 4, 5, 5, 6, 6, 6, 3, 3, 0, 2, 1, 1, 1, 1, 0],
 [5, 5, 5, 5, 5, 5, 6, 6, 6, 2, 2, 2, 2, 2, 2, 0, 0, 0],
 [4, 4, 5, 5, 5, 5, 6, 6, 6, 3, 1, 1, 1, 2, 2, 0, 0, 0],
 [3, 4, 4, 5, 5, 5, 5, 6, 7, 1, 2, 2, 2, 1, 2, 0, 0, 0],
 [3, 3, 4, 4, 4, 5, 5, 6, 6, 3, 2, 1, 0, 2, 1, 0, 1, 0],
 [2, 3, 4, 5, 5, 5, 5, 5, 6, 3, 3, 1, 1, 0, 2, 1, 0, 1],
 [5, 5, 5, 5, 5, 5, 6, 6, 6, 2, 2, 2, 2, 2, 2, 0, 0, 0],
 [4, 4, 5, 5, 5, 5, 6, 6, 6, 3, 1, 1, 1, 2, 2, 0, 0, 0],
 [2, 3, 4, 5, 5, 5, 5, 5, 6, 3, 3, 1, 1, 0, 2, 1, 0, 1],
 [5, 5, 5, 5, 5, 5, 6, 6, 6, 2,

In [551]:
np.unique(np.array(n_atoms_second_fourth_shell),axis=0,return_counts=True,return_index=True)

(array([[2, 3, 4, 5, 5, 5, 5, 5, 6, 3, 2, 1, 1, 0, 2, 1, 0, 0],
        [2, 3, 4, 5, 5, 5, 5, 5, 6, 3, 3, 1, 1, 0, 2, 1, 0, 1],
        [2, 4, 4, 4, 5, 5, 6, 6, 6, 3, 2, 0, 2, 1, 1, 1, 0, 0],
        [3, 3, 4, 4, 4, 5, 5, 6, 6, 3, 2, 1, 0, 2, 1, 0, 1, 0],
        [3, 3, 4, 4, 5, 5, 5, 5, 6, 2, 2, 1, 1, 1, 1, 0, 0, 0],
        [3, 3, 4, 4, 5, 5, 6, 6, 6, 2, 2, 0, 2, 1, 1, 0, 0, 0],
        [3, 3, 4, 4, 5, 5, 6, 6, 6, 3, 3, 0, 2, 1, 1, 1, 1, 0],
        [3, 4, 4, 5, 5, 5, 5, 6, 7, 1, 2, 2, 2, 1, 2, 0, 0, 0],
        [4, 4, 5, 5, 5, 5, 6, 6, 6, 3, 1, 1, 1, 2, 2, 0, 0, 0],
        [5, 5, 5, 5, 5, 5, 6, 6, 6, 2, 2, 2, 2, 2, 2, 0, 0, 0]]),
 array([ 5,  7,  2,  6,  0,  1,  3, 11, 10,  9]),
 array([  12,   74,    5,   35,    1,    1,  155, 2466,  804,  470]))

In [555]:
np.unique(np.array(n_atoms_second_shell),axis=0,return_counts=True,return_index=True)

(array([[2, 3, 4, 5, 5, 5, 5, 6, 5],
        [2, 3, 5, 4, 5, 5, 5, 5, 6],
        [2, 4, 5, 5, 4, 4, 6, 6, 6],
        [3, 3, 4, 4, 6, 5, 5, 5, 5],
        [3, 3, 5, 5, 4, 4, 6, 6, 6],
        [3, 3, 6, 4, 4, 4, 5, 5, 6],
        [4, 5, 5, 4, 5, 5, 6, 6, 6],
        [5, 4, 5, 3, 4, 5, 6, 5, 7],
        [5, 5, 5, 5, 5, 5, 6, 6, 6]]),
 array([ 7,  5,  2,  0,  1,  6, 10, 11,  9]),
 array([  74,   12,    5,    1,  156,   35,  804, 2466,  470]))

#### Std of number of atoms second shell

In [300]:
np.std(np.unique(np.sort(np.array(n_atoms_second_shell),axis=1),axis=0,return_counts=True)[0],axis=1)

array([1.16534316, 1.24721913, 1.06574034, 0.95581392, 1.15470054,
       1.09994388, 0.73702773, 0.47140452])

From these subgroups find how many have the same distances

#### Sum distribution, average distance

In [450]:
sum_distances = []
average_distances = []
std_distances = []
centroid_dev = []
single_dist_final = []
single_dist_all_sorted = []
single_dist_final_flat = []
for n,structure in enumerate(structures):
    average_distances_atom = []
    sum_distances_atom = []
    centroid_dev_atom = []
    single_dist_atom = []
    single_dist_atom_ext = []
    #std_distances_atom = []
    ti_dist_ext = []
    for i in range(len(ti_atoms[n])):
        dist = 0
        atom_coord = []
        ti_dist = []

        for j in ti_second_shell_atom[n][i]:
            atom_coord.append(structure.cart_coords[j].tolist())
            single_dist = []
            for k in ti_second_shell_atom[n][i]:
                if j != k:
                    single_dist.append(round(structure.sites[j].distance(structure.sites[k]),6))
                    dist += round(structure.sites[j].distance(structure.sites[k]),6)
            single_dist.sort()
            #print(single_dist)
            ti_dist.append(single_dist)
            ti_dist_ext.extend(single_dist)
        ti_dist.sort()
        ti_dist_ext.extend([item for sublist in ti_dist for item in sublist])
        #print(ti_dist)
        center = centroidnp(np.array(atom_coord))
        '''centroid_dev_atom.append(\
            np.sqrt(
                ((center-structure.cart_coords[j])[0])**2 \
            + ((center-structure.cart_coords[j])[1])**2 \
            + ((center-structure.cart_coords[j])[2])**2)
        )'''
        #std.append(np.std(atom_coord,axis=0))
        average_distances_atom.append(dist/len(ti_second_shell_atom[n][i]))
        sum_distances_atom.append(dist)
        single_dist_atom.append(ti_dist)
        single_dist_atom_ext.extend(ti_dist)
        #####TEST
        sorted_indices = np.argsort(n_atoms_second_shell[n])
        single_dist_sorted = []
        for i in sorted_indices:
            single_dist_sorted.extend(ti_dist_ext)
        #print(np.sort(np.sort(single_dist_final[n][i]),axis=0))
        single_dist_all_sorted.append(single_dist_sorted)

        ####
    sum_distances.append(sum_distances_atom)
    average_distances.append(average_distances_atom)
    centroid_dev.append(centroid_dev_atom)
    single_dist_final.append(single_dist_atom) #THIS ONE
    #print(ti_dist_ext)
    single_dist_final_flat.append(ti_dist_ext) #THIS ONE
    #std_distances.append(std)
        #print(i,j,len(ti_second_shell_atom[n][i]),dist)
#print(len(single_dist_all_sorted))
#print(single_dist_final_flat)

In [410]:
single_dist_final[0]

[[[3.595992, 5.0855], [3.595992, 6.22844], [5.0855, 6.22844]],
 [[3.595992, 5.0855], [3.595992, 6.22844], [5.0855, 6.22844]],
 [[3.595992, 3.595992, 3.595992],
  [3.595992, 6.22844, 6.22844],
  [3.595992, 6.22844, 6.22844],
  [3.595992, 6.22844, 6.22844]],
 [[3.595992, 3.595992, 3.595992],
  [3.595992, 6.22844, 6.22844],
  [3.595992, 6.22844, 6.22844],
  [3.595992, 6.22844, 6.22844]],
 [[3.595992, 3.595992, 3.595992, 6.22844, 6.22844],
  [3.595992, 3.595992, 3.595992, 6.22844, 6.22844],
  [3.595992, 3.595992, 3.595992, 6.22844, 6.22844],
  [3.595992, 3.595992, 3.595992, 6.22844, 6.22844],
  [3.595992, 3.595992, 3.595992, 6.22844, 6.22844],
  [3.595992, 3.595992, 3.595992, 6.22844, 6.22844]],
 [[3.595992, 3.595992, 3.595992, 6.22844],
  [3.595992, 3.595992, 6.22844, 6.22844],
  [3.595992, 5.0855, 6.22844, 6.22844],
  [3.595992, 5.0855, 6.22844, 6.22844],
  [3.595992, 6.22844, 6.22844, 6.22844]],
 [[3.595992, 3.595992, 3.595992, 6.22844],
  [3.595992, 3.595992, 6.22844, 6.22844],
  [3.59

In [418]:
np.where(np.sort(np.array(n_atoms_second_shell),axis=1) == unique_coord[0])

(array([   0,    0,    0, ..., 4022, 4022, 4022]),
 array([1, 2, 4, ..., 4, 5, 6]))

In [435]:
a = np.sort(np.array(n_atoms_second_shell),axis=1)
#np.all(np.sort(np.array(n_atoms_second_shell),axis=1) == unique_coord[0])
np.where((a == unique_coord[3]).all(axis=1))[0]
np.where(np.all(a == unique_coord[0],axis=1))

(array([   5,    7,   13,   16,   25,   27,   30,   45,   47,   50,   66,
          68,   71,   74,   76,   82,   87,  103,  108,  123,  160,  169,
         225,  227,  233,  236,  246,  251,  276,  314,  316,  343,  405,
         418,  479,  499,  500,  503,  512,  516,  521,  535,  556,  558,
         568,  574,  578,  689,  693,  698,  712,  740,  774,  884,  886,
         888,  897,  905,  910,  935,  949,  954,  969, 1095, 1100, 1118,
        1133, 1392, 1475, 1497, 1702, 1717, 1738, 2052, 2054, 2058, 2066,
        2287, 2400, 2406, 2411, 2443, 2497, 2508, 2621, 3892]),)

### Find structures belonging to classes based on unique_coordination

In [446]:
structure_groups = []
for unique in unique_coord:
    structure_groups.append(np.where(np.all(np.sort(np.array(n_atoms_second_shell),axis=1) == unique,axis=1))[0].tolist())

In [451]:
len(single_dist_final_flat)

4023

In [454]:
coord_group_all = []
for structure_group in structure_groups:
    coord_group = []
    for n in structure_group:
        coord_group.append(single_dist_final_flat[n])
    coord_group_all.append(coord_group)


In [524]:
np.array(coord_group_all[0][4]) - np.array(coord_group_all[0][0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [509]:
np.unique(np.array(coord_group_all[0]),axis=0)
unique_coord_dist = np.unique(np.array(coord_group_all[1]),axis=0)#,return_counts=True,return_index=True)
len(unique_coord_dist)

1

In [232]:
np.where(np.all(np.sort(np.round(sum_distances,6),axis=1)==np.sort(np.round(sum_distances,6),axis=1)[0]))
sum_distances_sorted = np.sort(np.round(sum_distances,6),axis=1)
average_distances_sorted = np.sort(np.round(average_distances,6),axis=1)

In [231]:
np.unique(sum_distances_sorted,axis=0, return_counts=True)

(array([[  7.191984,  51.395816,  53.681696,  53.681696,  92.979424,
          92.979424, 140.162272, 142.448152, 143.141288],
        [ 10.171   ,  29.819864,  53.681696,  92.979424,  92.979424,
          92.979424,  93.67256 ,  98.937456, 145.427168],
        [ 12.45688 ,  29.819864,  53.681696,  92.979424,  92.979424,
          92.979424,  95.95844 ,  98.937456, 145.427168],
        [ 24.554968,  24.554968,  53.681696,  53.681696,  92.979424,
          92.979424, 140.855408, 145.427168, 145.427168],
        [ 24.554968,  26.840848,  58.946592,  58.946592,  58.946592,
          98.937456, 101.223336, 139.469136, 148.406184],
        [ 26.840848,  26.840848,  53.681696,  53.681696,  92.979424,
          92.979424, 145.427168, 145.427168, 145.427168],
        [ 26.840848,  51.395816,  53.681696,  88.407664,  88.407664,
          90.693544,  95.95844 , 145.427168, 196.822984],
        [ 29.819864,  29.819864,  58.946592,  58.946592, 101.223336,
         101.223336, 101.223336, 101.22333

In [233]:
np.unique(average_distances_sorted,axis=0, return_counts=True)

(array([[ 3.595992, 12.848954, 13.420424, 13.420424, 18.595885, 18.595885,
         23.360379, 23.741359, 23.856881],
        [ 5.0855  ,  9.939955, 13.420424, 18.595885, 18.595885, 18.595885,
         18.734512, 19.787491, 24.237861],
        [ 6.22844 ,  9.939955, 13.420424, 18.595885, 18.595885, 18.595885,
         19.191688, 19.787491, 24.237861],
        [ 8.184989,  8.184989, 13.420424, 13.420424, 18.595885, 18.595885,
         23.475901, 24.237861, 24.237861],
        [ 8.184989,  8.946949, 14.736648, 14.736648, 14.736648, 19.787491,
         20.244667, 23.244856, 24.734364],
        [ 8.946949,  8.946949, 13.420424, 13.420424, 18.595885, 18.595885,
         24.237861, 24.237861, 24.237861],
        [ 8.946949, 12.848954, 13.420424, 17.681533, 17.681533, 18.138709,
         19.191688, 24.237861, 28.117569],
        [ 9.939955,  9.939955, 14.736648, 14.736648, 20.244667, 20.244667,
         20.244667, 20.244667, 23.244856],
        [13.420424, 13.420424, 17.681533, 17.681533, 18.

In [132]:
magic_index = n_atoms_second_shell-(np.array(sum_distances)/np.amax(sum_distances[1]))

In [121]:
magic_index

array([[3.34072969, 1.87777363, 1.87777363, 3.34072969, 1.92943257,
        1.87777363, 1.92943257, 3.34072969, 3.34072969],
       [3.41481758, 2.72983774, 2.75906772, 2.72983774, 1.90020259,
        2.75906772, 4.10331786, 3.45967549, 4.        ],
       [3.45967549, 3.45967549, 3.45967549, 3.45967549, 3.45967549,
        3.45967549, 3.45967549, 3.45967549, 3.45967549]])

In [166]:
magic_index[0]

array([2.79494984, 2.79494984, 3.59466589, 3.59466589, 5.04096918,
       4.30395856, 4.30395856, 4.30395856, 4.30395856])

In [103]:
n_atoms_second_shell

[[4, 2, 2, 4, 2, 2, 2, 4, 4],
 [4, 3, 3, 3, 2, 3, 5, 4, 5],
 [4, 4, 4, 4, 4, 4, 4, 4, 4]]

In [173]:
a = np.sum(magic_index,axis=1) 

In [171]:
np.where(np.around(a,6) == 41.569715)

(array([   9,   14,   17,   20,   28,   32,   35,   40,   48,   52,   60,
          72,   73,   78,   84,   89,   93,   96,  105,  110,  114,  117,
         125,  129,  132,  139,  142,  148,  158,  161,  164,  171,  175,
         178,  185,  188,  194,  205,  210,  216,  234,  242,  248,  255,
         258,  265,  268,  278,  282,  285,  295,  305,  318,  322,  325,
         332,  335,  345,  348,  353,  356,  362,  372,  381,  385,  392,
         393,  407,  411,  413,  422,  426,  432,  441,  442,  450,  457,
         467,  472,  484,  486,  508,  513,  518,  526,  529,  536,  539,
         544,  549,  560,  564,  570,  575,  579,  583,  586,  593,  597,
         601,  603,  610,  613,  616,  622,  625,  631,  668,  674,  677,
         685,  690,  695,  700,  704,  707,  713,  716,  722,  727,  733,
         735,  742,  746,  749,  756,  759,  765,  776,  780,  782,  789,
         795,  803,  806,  812,  822,  838,  841,  847,  857,  871,  890,
         907,  914,  917,  924,  927, 

In [172]:
view(AseAtomsAdaptor().get_atoms(structures[14]))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/cc/bin/p...>

In [134]:
np.unique(np.sum(magic_index,axis=1))

array([35.03603486, 35.074705  , 35.08193836, 35.11337513, 36.61389883,
       36.65256896, 36.67677236, 38.24016959, 39.94344441, 41.56971517])

In [136]:
np.where(np.sum(magic_index,axis=1) == 41.56971517)

(array([], dtype=int64),)

In [52]:
np.sort(sum_distances[0])

array([ 3.595992,  3.595992,  6.22844 ,  6.22844 ,  6.22844 , 16.797626,
       16.797626, 16.797626, 16.797626])

In [13]:
#second_coord_max

In [79]:
#Old one
second_coord_max = []
second_coord_min = []
for n,structure in enumerate(structures[0:1]):
    sum_coord = []    
    for i in ti_atoms[n]:
        sum_coord_atom = 0
        for j in ti_atoms[n]:
            sum_coord_atom += (round(structure.distance_matrix[i][j],6) == round(shells[2],6))
        sum_coord.append(sum_coord_atom)
    second_coord_max.append(max(sum_coord))
    second_coord_min.append(min(sum_coord))

#### Fourth shell

In [ ]:
fourth_coord_max = []
for n,structure in enumerate(structures):
    sum_coord = []    
    for i in ti_atoms[n]:
        sum_coord_atom = 0
        for j in ti_atoms[n]:
            sum_coord_atom += (round(structure.distance_matrix[i][j],6) == round(shells[2],6))
            sum_coord_atom += (round(structure.distance_matrix[i][j],6) == round(shells[4],6))
        sum_coord.append(sum_coord_atom)
    fourth_coord_max.append(max(sum_coord))

### Standard deviation

#### Cartesian coordinates

In [175]:
std = [] 
std_sum = []
std_std = []
for n,structure in enumerate(structures[0:10000]):
    ti_coord = []
    for i in ti_atoms[n]:
        ti_coord.append(structure.cart_coords[i].tolist())
    
    std.append(np.std(ti_coord,axis=0).tolist())
    std_sum.append(np.round(np.sum(ti_coord),6))
    std_std.append(np.std(np.std(ti_coord,axis=0)))
display(np.unique(std))
display(np.unique(std_sum))
display(np.unique(std_std))
        

array([1.74161819, 1.99777308, 2.07614668, 2.32978374, 2.32978374,
       2.39732769, 2.68029384, 2.70991125, 2.79688231, 2.96317633,
       3.1206213 , 3.17136644, 3.17136644, 3.27049548, 3.41379613])

array([-45.7695, -40.684 , -35.5985, -30.513 , -20.342 ])

array([0.36488537, 0.37306871, 0.40047482, 0.44463808, 0.46387399,
       0.48123779, 0.49585539, 0.54092928, 0.55323722, 0.69852801])

#### Spherical coordinates

In [100]:
distribution_com = [] 
for n,structure in enumerate(structures[0:100]):
    ti_coord = []
    ti_coord_sph = []
    for i in ti_atoms[n]:
        ti_coord.append(structure.cart_coords[i].tolist())
    centroid = centroidnp(np.array(ti_coord))
    ti_coord = ti_coord - centroid
    for coord in ti_coord:
        ti_coord_sph.append(cart2sph(coord[0],coord[1],coord[2]))
    print(np.std(ti_coord_sph,axis=0))
        

[1.69438047 0.46872947 1.08773498]
[1.70164327 0.66511309 1.44476935]
[1.87435127 0.61040919 1.52925174]


### Planes

In [198]:
slab = SlabGenerator(SpacegroupAnalyzer(structures[300]).get_conventional_standard_structure(),[1,0,0],10,10).get_slab()
#view(AseAtomsAdaptor().get_atoms(SpacegroupAnalyzer(structures[3000]).get_conventional_standard_structure()))
#view(AseAtomsAdaptor().get_atoms(slab))
len(slab)

108

### Distance periodic site

In [59]:
coordination = [] 
for n,structure in enumerate(structures):
    second_coord = 0
    for i in ti_atoms[n]:        
        for j in ti_atoms[n]:
            if i != j:
                if round(structure.sites[i].distance(structure.sites[j]),6) == round(shells[2],6):
                    second_coord += 1
    #coordination.append(second_coord/len(ti_atoms[n]))
    coordination.append(second_coord)
                
coordination      

[26, 32, 36]

### Scan along row

#### Shells

In [91]:
def cart2sph(x, y, z):
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.arctan2(z, hxy)
    az = np.arctan2(y, x)
    if np.around(az,6) ==  np.around(2*np.pi,6) \
    or np.around(az,6) ==  -np.around(2*np.pi,6):
        az = 0.
    return [round(az,6), round(el,6), round(r,6)]

In [ ]:
def cart2cent2sph(atom,center):
    x = atom[0]-center[0]
    y = atom[1]-center[1]
    z = atom[2]-center[2]
    return cart2sph(x, y, z)

In [ ]:
def cart2sin(x, y, z):
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.sin(np.arctan2(z, hxy))
    az = np.cos(np.arctan2(y, x))
    return [round(az,6), round(el,6), round(r,6)]

In [65]:
def centroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return np.array([sum_x/length, sum_y/length, sum_z/length])

In [ ]:
#Spherical coordinates
#Make sure there are no gaps even if they have the same polar and azimuthal coords
max_row = []
for n,structure in enumerate(structures[3000:3001]):
    for i in ti_atoms[n]:
        spherical_coord = []
        row = []
        for j in ti_atoms[n]:
            #print(i,j)
            center_cart = structure.cart_coords[j]-structure.cart_coords[i]
            spherical_coord.append(cart2sph(center_cart[0],center_cart[1],center_cart[2]))   
        spherical_coord = np.around(np.array(spherical_coord),6)
        #print(spherical_coord[:,0:1],spherical_coord[:,0:1]+np.pi)
        for el in spherical_coord:
            #print(el[0:3],spherical_coord[:,0:3])
            #print(spherical_coord[:,0:2] == el[0:2],np.sum(np.prod(spherical_coord[:,0:2] == el[0:2],axis=1)))
            #print(np.sum(np.multiply(spherical_coord[:,0] == el[0]+np.pi,spherical_coord[:,1] == -el[1])))
            row.append(np.sum(np.prod(spherical_coord[:,0:2] == el[0:2],axis=1)))#+
                 #np.sum(np.multiply(spherical_coord[:,0] == el[0]+np.pi,spherical_coord[:,1] == -el[1])))
    max_row.append(max(row)+1)       
       

In [ ]:
#Sinusoidal coordinates
max_row = []
for n,structure in enumerate(structures[3000:3001]):
    for i in ti_atoms[n]:
        spherical_coord = []
        row = []
        #print(i)
        for j in ti_atoms[n]:
            #print(i,j)
            center_cart = structure.cart_coords[j]-structure.cart_coords[i]
            spherical_coord.append(cart2sin(center_cart[0],center_cart[1],center_cart[2]))   
        spherical_coord = np.around(np.array(spherical_coord),6)
        #print(spherical_coord)
        for el in spherical_coord:
            #print(spherical_coord[:,0:3])
            #print(spherical_coord[:,0:2] == el[0:2],np.sum(np.prod(spherical_coord[:,0:2] == el[0:2],axis=1))) 
            row.append(np.sum(np.prod(spherical_coord[:,0:2] == el[0:2],axis=1))+
                 np.sum(np.prod(spherical_coord[:,0:2] == -el[0:2],axis=1)))
    max_row.append(max(row)+1)       
       

In [ ]:
#get neighbours
max_radius = max(np.sum(structure.lattice.matrix,axis=1))
for n,structure in enumerate(structures[0:1]):
    for i in ti_atoms[n]:
        #print(structure.get_neighbor_list(max_radius)[3])
        for neigh in structure.get_neighbors(structure.sites[i],max_radius):
            if neigh.as_dict()['species'][0]['element'] == 'Ti':
                
        #for j in ti_atoms[n]:
            #if i != j:
                #print(structure.get_neighbors(structure.sites[i],3.))
#len(structure.get_all_neighbors(10.)[0])

# Neighbour analysis and classification

In [ ]:
shells

In [ ]:
ti_neigh_list = []
for i,structure in enumerate(structures[0:1]):
    for ti_atom_1 in ti_atoms[i]:
        ti_neigh = []
        for ti_atom_2 in ti_atoms[i]:
            if structure.distance_matrix[ti_atom_1][ti_atom_2] - shells[2] < 0.001:
                ti_neigh.append(cart2cent2sph(structure.cart_coords[ti_atom_2],structure.cart_coords[ti_atom_1]))
        ti_neigh_list.append(ti_neigh)
ti_neigh_list

In [ ]:
structures[0]

In [ ]:
structures[0].distance_matrix

### Fourier transform

In [ ]:
ti_coord = []
for i in ti_atoms[0]:
    ti_coord.append(structures[0].sites[i].coords.tolist())
np.fft.fftn(np.array(ti_coord))

In [ ]:
df = pd.DataFrame(list(zip(space_group,average_ti,second_coord_sum,second_coord_max,fourth_coord_max)),
               columns =['Space Group','Average Ti distance','Sum second coord','Max second coord','Max fourth coord'])

df[df['Space Group'] == 1]
df